In [1]:
import pymongo
from pymongo import MongoClient
import pandas as pd

## Connection to the mongo file

In [2]:
client = pymongo.MongoClient("mongodb://localhost:27017/")


In [3]:
client.drop_database('movies_database')

In [4]:
db = client["movies_database"]

## Movies Collection

In [5]:
db.movies.drop()
movies_collection=db.movies

In [6]:
movies_collection.insert_many([
    {"movie_id": 1, "title": "Inception", "genre": "Sci-Fi", "release_year": 2010},
    {"movie_id": 2, "title": "The Shawshank Redemption", "genre": "Drama", "release_year": 1994},
    {"movie_id": 3, "title": "Toy Story", "genre": "Fiction", "release_year": 1995},
    {"movie_id": 4, "title": "The Truman Show", "genre": "Thriller", "release_year": 1999},
    {"movie_id": 5, "title": "Star-Wars", "genre": "Sci-Fi", "release_year": 1977}
])

InsertManyResult([ObjectId('655d37d64f9c3c89cc96f078'), ObjectId('655d37d64f9c3c89cc96f079'), ObjectId('655d37d64f9c3c89cc96f07a'), ObjectId('655d37d64f9c3c89cc96f07b'), ObjectId('655d37d64f9c3c89cc96f07c')], acknowledged=True)

## User Collection

In [7]:
db.users.drop()
movies_users=db.users

In [8]:
movies_users.insert_many([
    { "user_id": 101,"username": "Justin", "email": "justin@example.com", "age": 22, "gender": "Male"},
    { "user_id": 102,"username": "Joseph", "email": "joseph@example.com", "age": 22, "gender": "Male"},
    { "user_id": 103,"username": "Satyaprakash", "email": "satyaprakash@example.com", "age": 26, "gender": "Male"},
    { "user_id": 104,"username": "Shrey", "email": "shrey@example.com", "age": 23, "gender": "Male"},
    { "user_id": 105,"username": "Ron", "email": "ron@example.com", "age": 55, "gender": "Male"}
])

InsertManyResult([ObjectId('655d37d64f9c3c89cc96f07d'), ObjectId('655d37d64f9c3c89cc96f07e'), ObjectId('655d37d64f9c3c89cc96f07f'), ObjectId('655d37d64f9c3c89cc96f080'), ObjectId('655d37d64f9c3c89cc96f081')], acknowledged=True)

## Ratings Collection

In [9]:
db.ratings.drop()
movies_ratings=db.ratings

In [10]:
movies_ratings.insert_many([
    {"receipt_id": 1001, "r_user_id": 101, "r_movie_id": 5, "rating": 6},
    {"receipt_id": 1002, "r_user_id": 102, "r_movie_id": 4, "rating": 9.5},
    {"receipt_id": 1003, "r_user_id": 103, "r_movie_id": 3, "rating": 8},
    {"receipt_id": 1004, "r_user_id": 104, "r_movie_id": 2, "rating": 4.5},
    {"receipt_id": 1005, "r_user_id": 105, "r_movie_id": 1, "rating": 6.5},
    {"receipt_id": 1006, "r_user_id": 101, "r_movie_id": 3, "rating": 7}
])

InsertManyResult([ObjectId('655d37d64f9c3c89cc96f082'), ObjectId('655d37d64f9c3c89cc96f083'), ObjectId('655d37d64f9c3c89cc96f084'), ObjectId('655d37d64f9c3c89cc96f085'), ObjectId('655d37d64f9c3c89cc96f086'), ObjectId('655d37d64f9c3c89cc96f087')], acknowledged=True)

## Location Collection

In [11]:
db.location.drop()
movies_location=db.location

In [12]:
movies_location.insert_many([
    {"country":"USA", "city":"New York", "l_movie_id":3},
    {"country":"AUS", "city":"Sydney", "l_movie_id":5},
    {"country":"UK", "city":"Leeds", "l_movie_id":2},
    {"country":"India", "city":"Mumbai", "l_movie_id":4},
    {"country":"Germany", "city":"Berlin", "l_movie_id":1}

])

InsertManyResult([ObjectId('655d37d64f9c3c89cc96f088'), ObjectId('655d37d64f9c3c89cc96f089'), ObjectId('655d37d64f9c3c89cc96f08a'), ObjectId('655d37d64f9c3c89cc96f08b'), ObjectId('655d37d64f9c3c89cc96f08c')], acknowledged=True)

##  Query with 2 collections

In [13]:
# This query joins Movie collection & Location collections

query1 = movies_collection.aggregate(
    [ { '$lookup': { 'from':         'location',
                     'localField':   'movie_id',
                     'foreignField': 'l_movie_id',
                     'as':           'movie_id_doc'
                   }
      },
      { '$unwind': '$movie_id_doc' },
      { '$project': { '_id': 0,
                      'movie_id': 1,
                      'title': 1,
                       'country':'$movie_id_doc.country'
                    }
      }             
    ]
)

pd.DataFrame(query1)

,movie_id,title,country
0,1,Inception,Germany
1,2,The Shawshank Redemption,UK
2,3,Toy Story,USA
3,4,The Truman Show,India
4,5,Star-Wars,AUS


## Query with 3 collections

In [14]:
# This query joins ratings collection, users collection & movies collection

query2= movies_ratings.aggregate([ 
    { '$lookup': { 'from':         'movies',
                     'localField':   'r_movie_id',
                     'foreignField': 'movie_id',
                     'as':           'movie_id_doc'
                   }
      },
    
      { '$unwind': '$movie_id_doc' },
    
      {'$lookup': { 'from':         'users',
                     'localField':   'r_user_id',
                     'foreignField': 'user_id',
                     'as':           'user_id_doc'
                   }
      },
    
      { '$unwind': '$movie_id_doc' },
    
      { '$project': { '_id': 0,
                      'receipt_id': 1,
                      'title': "$movie_id_doc.title",
                      'username':"$user_id_doc.username",
                      'rating':1
                       
                    }
      }             
    ]
)
pd.DataFrame(query2)

,receipt_id,rating,title,username
0,1001,6.0,Star-Wars,[Justin]
1,1002,9.5,The Truman Show,[Joseph]
2,1003,8.0,Toy Story,[Satyaprakash]
3,1004,4.5,The Shawshank Redemption,[Shrey]
4,1005,6.5,Inception,[Ron]
5,1006,7.0,Toy Story,[Justin]


## Query with 4 collections

In [15]:
#This query joins all collection(movies, ratings,users, location) and it only shows results where the movie title is "Toy Story"
query3=movies_ratings.aggregate([ 
    { '$lookup': { 'from':         'movies',
                     'localField':   'r_movie_id',
                     'foreignField': 'movie_id',
                     'as':           'movie_id_doc'
                   }
      },
    
      { '$unwind': '$movie_id_doc' },
    
      {'$lookup': { 'from':         'users',
                     'localField':   'r_user_id',
                     'foreignField': 'user_id',
                     'as':           'user_id_doc'
                   }
      },
    
      { '$unwind': '$user_id_doc' },
    
    {'$lookup': { 'from':         'location',
                     'localField':   'r_movie_id',
                     'foreignField': 'l_movie_id',
                     'as':           'location_doc'
                   }
      },
    
      { '$unwind': '$location_doc' },
      { '$match' : { '$and': [ {'movie_id_doc.title': 'Toy Story'}
                            
                             ]
                   }
      },
      { '$project': { '_id': 0,
                      'receipt_id': 1,
                      'title': "$movie_id_doc.title",
                      'username':"$user_id_doc.username",
                      'rating':1,
                     'country':'$location_doc.country',
                     'city':'$location_doc.city'
                       
                    }
      }             
    ]
)
pd.DataFrame(query3)

,receipt_id,rating,title,username,country,city
0,1003,8,Toy Story,Satyaprakash,USA,New York
1,1006,7,Toy Story,Justin,USA,New York
